Checking that missing dates are actually missing like we think

In [ ]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

Checking in grid outlooks

In [ ]:
missing_dates = ['200204250000', '200208300000', '200304150000', '200304160000', '200306250000', '200307270000', '200307280000', '200312280000', '200404140000', '200408090000', '200905280000', '201105210000', '202005240000']

In [ ]:
grid_outlook_location = 'data/outlooks/grid_outlooks.nc'
grid_outlooks = xr.open_dataset(grid_outlook_location)

In [ ]:
max_outlooks = grid_outlooks.max(dim = ['x', 'y'])['prob']

dates = max_outlooks['time']
thresholds = {'': .15,
              ' Tornado': .05,
              ' Wind': .15,
              ' Hail': .15}
days = ['Day 3', 'Day 2 7', 'Day 2 17', 'Day 1']
hazard_days = ['Day 2 7', 'Day 2 17', 'Day 1']
investigate_dates = []
for date in dates:
    max_outlooks_date = max_outlooks.sel(time = date)
    for hazard in ['', ' Tornado', ' Wind', ' Hail']:
        if hazard == '':
            this_days = days
        else:
            this_days = hazard_days
        outlook_keys = [item + hazard for item in this_days]
        max_prob = max_outlooks_date.sel(outlook = outlook_keys).max()
        if max_prob >= thresholds[hazard]:
            for outlook_key in outlook_keys:
                if max_outlooks_date.sel(outlook = outlook_key) == 0 and outlook_key == 'Day 1':
                    print(str(date.data) + ' ' + str(outlook_key))
                    investigate_dates.append(str(date.data))

                

print(investigate_dates)

: 

In [ ]:
print(missing_dates)
print(investigate_dates)
len(investigate_dates)

In [ ]:
pph = xr.open_dataset('data/pph/labelled_pph.nc')
investigate_pph = pph.sel(time = investigate_dates)

In [ ]:
mdt_investigate_dates = investigate_pph['MAX_CAT'].isin(['MDT', 'HIGH'])
mdt_investigate_dates
mdt_investigate_pph = investigate_pph.sel(time = mdt_investigate_dates)
mdt_investigate_pph

Checking in raw data

In [ ]:
raw_dir = '~/Downloads'
outlook_raw_location = raw_dir + '/outlooks'

# Read convective outlooks into outlooks
year_list = [[1987, 1991], [1992, 1999], [2000, 2007], [2008, 2015], [2016, 2023]]
outlook_list = []

for years, i in zip(year_list, range(len(year_list))):
    print('reading file ' + str(i) + ', years ' + str(years[0]) + '-' + str(years[1]))
    gdf = gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359')
    outlook_list.append(gdf)

# Concatenate all GeoDataFrames into one
outlooks_original = pd.concat(outlook_list, ignore_index=True)

print('files read')
    
outlooks_original

In [ ]:
for date in investigate_dates:
    print(date)
    print(outlooks_original[(outlooks_original['ISSUE'] == date[0:8] + '1200') & (outlooks_original['DAY'] == 1)]['CYCLE'].unique())


In [ ]:
outlooks_original[(outlooks_original['ISSUE'] == '202005241200')& (outlooks_original['DAY'] == 1)]

Look for dates with day 2 17 but no day 1 6 outlooks--there are a lot more, but with some spot checking they generally seem to have actually happened like this

In [ ]:
data_location = 'data'
# outlooks, pph, reports = read_datasets(data_location)
outlook1 = gp.read_file(f"{data_location}/outlooks/all_outlooks_1.shp", engine="pyogrio")
outlook2 = gp.read_file(f"{data_location}/outlooks/all_outlooks_2.shp", engine="pyogrio")
outlooks = gp.GeoDataFrame(pd.concat([outlook1, outlook2], ignore_index=True), crs=outlook1.crs)

In [ ]:
# Pre-filter relevant cycles to speed things up
outlook_subset = outlooks[(outlooks['CYCLE'].isin([6, 17])) & outlooks['CATEGORY'].isin(['CATEGORICAL', 'WIND', 'HAIL', 'TORNADO'])]

# Create a pivot table with counts of each cycle per (CATEGORY, DATE)
pivot = outlook_subset.pivot_table(index=['DATE'], 
                                   columns='CYCLE', 
                                   aggfunc='size', 
                                   fill_value=0)



# Check where 17 exists and 6 does not
condition =  (pivot.get(17, 0) > 0) & (pivot.get(6, 0) == 0)

for date in pivot.index[condition]:
    print(date)

len(pivot.index[condition])

In [ ]:
['200204250000', '200208300000', '200304150000', '200304160000', '200306250000', '200307270000', '200307280000', '200312280000', '200404140000', '200408090000', '200905280000', '201105210000', '202005240000']